In [ ]:
import json
import sys
#This script merge doccano output file and raw dataset
#Start and ending index
QSTART = 615
QLEN = 486

In [ ]:
# saves dictionary as json file
def save_as_json(file_name, data):
    with open(file_name, "w") as formatted:
        json.dump(data, formatted)
        
# convert question texts to dictionaries
def question_texts_to_dict(questions, paragraph, id_):
    splitted = questions.split("\n")
    empty = True
    splitted_questions = []
    for i in range(int(len(splitted) / 2)):
        question = splitted[i*2]
        data = question.split(": ")  # Careful here. Did you have space after question names?
        # data = question.split(":")
        
        if data[1] != '':
            splitted_questions.append(data[1].strip())
            empty = False
            
    question_dict = {}
    question_dict["questions"] = splitted_questions
    #print(paragraph[0])
    question_dict["text"] = paragraph
    question_dict["id"] = id_
    return question_dict

# read files under questions directory
def read_questions_dir(path, file_count):
    questions_dict = []
    id_ = 1
    #print(file_count)
    for i in range(QSTART, QSTART+QLEN + 1):
        try:
            f1 = open(path + str(i) + ".txt", "r", encoding="utf-8")
            f2 = open(path + str(i) + "_soru.txt", "r", encoding="utf-8")
        except:
            print("Dosya " + str(i) + ".txt açılamıyor")
            continue
        paragraph = f1.read()
        questions = f2.read()
        paragraph = paragraph.replace("\n", '')

        if paragraph[0] == ' ':
            paragraph = paragraph[1:]
            
        #print(paragraph[0])
        try:
            dictionary = question_texts_to_dict(questions, paragraph, id_)
        except:
            print("Dosya " + str(i) + " sorunlu")
#             print(questions)
#             print(paragraph)
            print(sys.exc_info()[0])
            return
        questions_dict.append(dictionary)
        id_ += 1
        f1.close()
        f2.close()

    return questions_dict

In [ ]:
import os.path
path = "dataset/"
count = os.listdir(path)
print(len(count))
fileCount =(int) (len(count)/2)
questions = read_questions_dir(path, fileCount)
save_as_json("questionsV2.json", questions)

print("Paragraf sayısı:", len(questions))

In [ ]:
# reads formatted answers file
def read_json(file_name):
    with open(file_name, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
    return data

# clears unnecessary whitespaces before and after answers
def clear_the_answers(answers):
    for j in range(len(answers["data"])):
        answer = answers["data"][j]
        for i in range(len(answer["label"])):
            full_answer = answer["data"][answer["label"][i][0]:answer["label"][i][1]]
            k = 0
            while (full_answer[k] == ' ' and k < len(answer)):
                answer["label"][i][0] += 1
                k += 1
                
            k = len(full_answer) - 1    
            while (full_answer[k] == ' ' and k >= 0):
                answer["label"][i][1] -= 1
                k -= 1

# reads raw doccano output file and save as json.
def read_doccano_file(file_path):
    with open(file_path, "r", encoding="utf-8") as json_file:
        answers = json_file.read()
        
    text_answers =  answers.split("\n")
    answer_dict = []
    for answer in text_answers[:-1]:
        dictionary = json.loads(answer)
        answer_dict.append(dictionary)
        
    data = {"data": answer_dict}
    clear_the_answers(data)
    new_filename = file_path.split(".")[0] + "_formatted.json"
    
    save_as_json(new_filename, data)
    return data

In [ ]:
file_path = "admin.jsonl"

answers = read_doccano_file(file_path)

In [ ]:
file_path = "admin_formatted.json"

answers = read_json(file_path)

In [ ]:
# Counts matched questions and answers based on texts.
def matched_questions_and_answers(questions, answers):
    eslesti = 0
    for j in range(len(questions)):
        found = False
        i = 0
        while(i < len(answers["data"]) and not found):
            print(answers["data"][i]['data'],"\n",questions[j]["text"],"\n","\n")
            if answers["data"][i]['data'] == questions[j]["text"]:
                eslesti += 1
                found = True
            i += 1
        if not found:
            print("Questions " + str(j) + " eşleşemedi")

    print("Matched files: ", eslesti)

# creates q&a dictionary.
def create_q_a_dictionary(answer, question, id_):
    qas_list = []
    q_id = 0
    
    for i in range(len(question["questions"])):
        answer_name = "cevap" + str(i + 1)
        k = 0
        while (k < len(answer['label']) and answer['label'][k][2] != answer_name):
            k += 1
        try:
            start = answer['label'][k][0] 
            end = answer['label'][k][1]
        except:
            print(answer, k, answer_name, question)
            print(sys.exc_info()[0])
            return
            

        q_dict = {
            "id": q_id,
            "question": question["questions"][i],
            "answer": answer['data'][start:end],
            "answer_start": start,
            "answer_end": end
        }

        qas_list.append(q_dict)
        q_id += 1

    if qas_list:
        dictionary = {
            "id": id_,
            "text": answer['data'],
            "qas": qas_list
        }
        return dictionary
    else:
        return None

# merges q&a dictionaries.    
def merge_questions_and_answers(questions, answers):
    dataset = []
    id_ = 0
    for j in range(len(questions)):
        found = False
        i = 0
        while(i < len(answers["data"]) and not found):
            if answers["data"][i]['data'] == questions[j]["text"]:
                found = True
            else:
                i += 1
        if found:   
            answer = answers["data"][i]
            question = questions[j]
            dictionary = create_q_a_dictionary(answer, question, id_)
            if dictionary != None:
                id_ += 1
                dataset.append(dictionary)  
                
    final_dataset = {"data": dataset}
    return final_dataset

In [ ]:
questions[0]["questions"]

['Erkan Meriç kimdir?',
 'Erkan Meriç ne zaman doğmuştur?',
 'Erkan Meriç ilköğretim ve lise eğitimini nerede tamamlamıştır?',
 'Erkan Meriç hangi bölümden mezun olmuştur?',
 'Erkan Meriç Best Model of Turkey 2009 yarışmasında kaçıncı oldu?',
 'Erkan Meriç Best Model of the World yarışmasında ne ünvanı almıştır?']

In [ ]:
print(answers["data"][0]['data'])

Özel Kuvvetler Komutanlığı, Türk Silahlı Kuvvetlerinin değişik sınıf ve rütbelerdeki subay, astsubay ve uzman erbaşlardan oluşan, iç ve dış tehditlerin bertaraf edilmesine karşı her türlü arazi ve iklim şartlarında görev yapabilecek nitelikte üst düzey eğitime tabi tutularak yetiştirilmiş seçkin askerlerden oluşan birlik. Doğrudan Genelkurmay Başkanlığına bağlı olarak tugay seviyesinde 1992 yılında kurulmuştur. Türk Silahlı Kuvvetleri kıyafet yönetmeliğine göre bordo renkte bere kullanırlar. İlk olarak 27 Eylül 1952 yılında "Hususi ve Yardımcı Muharip Birlikleri" adıyla Amerika Birleşik Devletleri Ordusu Özel Harekât Kuvvetleri ortak eğitimleri ile kurulmuş, 7 Kasım 1953 yılında "Seferberlik Tetkik Kurulu" adını almıştır. 14 Aralık 1970 yılında ise "Özel Harp Dairesi" adını almıştır. 14 Nisan 1992 yılında Körfez Savaşı'ndan sonra dış tehlikelerin belirmesi ve Irak'ın kuzeyinden tehditler gelmesi nedeniyle 1992 yılında yerleşkesi ve yapısı değiştirilerek kurulmuştur. İlk olarak tugay se

In [ ]:
eslesti = 0
for j in range(len(answers["data"])):
    found = False
    i = 0
    while(i < len(questions) and not found):
        print(answers["data"][j]['data'][0]+"\n"+questions[i]["text"][0])
        if answers["data"][j]['data'] == questions[i]["text"]:
            eslesti += 1
            found = True
        i += 1
    if not found:
        print("Answers " + str(j) + " eşleşemedi")

In [ ]:
print(answers["data"][0]['data'] ,"\n"+ questions[0]["text"])


Özel Kuvvetler Komutanlığı, Türk Silahlı Kuvvetlerinin değişik sınıf ve rütbelerdeki subay, astsubay ve uzman erbaşlardan oluşan, iç ve dış tehditlerin bertaraf edilmesine karşı her türlü arazi ve iklim şartlarında görev yapabilecek nitelikte üst düzey eğitime tabi tutularak yetiştirilmiş seçkin askerlerden oluşan birlik. Doğrudan Genelkurmay Başkanlığına bağlı olarak tugay seviyesinde 1992 yılında kurulmuştur. Türk Silahlı Kuvvetleri kıyafet yönetmeliğine göre bordo renkte bere kullanırlar. İlk olarak 27 Eylül 1952 yılında "Hususi ve Yardımcı Muharip Birlikleri" adıyla Amerika Birleşik Devletleri Ordusu Özel Harekât Kuvvetleri ortak eğitimleri ile kurulmuş, 7 Kasım 1953 yılında "Seferberlik Tetkik Kurulu" adını almıştır. 14 Aralık 1970 yılında ise "Özel Harp Dairesi" adını almıştır. 14 Nisan 1992 yılında Körfez Savaşı'ndan sonra dış tehlikelerin belirmesi ve Irak'ın kuzeyinden tehditler gelmesi nedeniyle 1992 yılında yerleşkesi ve yapısı değiştirilerek kurulmuştur. İlk olarak tugay se

In [ ]:
print("Questions length: {}, Answers length: {}".format(len(questions), len(answers["data"])))
matched_questions_and_answers(questions, answers)

In [ ]:
dataset = merge_questions_and_answers(questions, answers)
print(len(dataset["data"]))

In [ ]:
save_as_json("wiki_output.json", dataset)